In [2]:
#imports and constants
import time
import math 
from urllib.request import urlretrieve as rx
import numpy as np
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline
EarthRadius = 6378137
MinLatitude = -85.05112878
MaxLatitude = 85.05112878
MinLongitude = -180
MaxLongitude = 180
MaxLevelofDetail=23

In [3]:
def Clip(n, minValue, maxValue):
    """
    Clips a number to the specified minimum and maximum values
    input number and its min&max range
    output clipped value in range min and max"""
    return min(max(n,minValue),maxValue)

def MapSize(levelofDetail):
    """
    Determines the map width and height (in pixels) at a specified level
    of detail //map width = map height = 256 * 2pow(level) pixels
    input level of detail 1 to 23
    output returns height=width of map"""
    return int(256 << levelofDetail)

def GroundResolution(latitude,levelofDetail):
    """ 
    Determines the ground resolution (in meters per pixel) at a specified
    latitude and level of detail.
    input latitude,level of detail
    output ground resolution in meters per pixel"""
    latitude=Clip(latitude,MinLatitude,MaxLatitude)
    return math.cos(latitude * math.pi / 180) * 2 * math.pi * EarthRadius / MapSize(levelofDetail)

def MapScale(latitude,levelofDetail,screenDpi):
    """
    Determines the map scale at a specified latitude, level of detail,
    and screen resolution.
    input latitude,levelofDetail,screenDpi
    output mapscale"""
    return GroundResolution(latitude,levelofDetail)*screenDpi/0.0254

def LatLongToPixelXY(latitude,longitude,levelofDetail):
    """
    Converts a point from latitude/longitude WGS-84 coordinates (in degrees)
    into pixel XY coordinates at a specified level of detail.
    input latitude,level of detail,screen dpi
    output pixelX, pixelY"""
    latitude = Clip(latitude, MinLatitude, MaxLatitude)
    longitude = Clip(longitude, MinLongitude, MaxLongitude)
    
    x=(longitude+180)/360
    sinLatitude=math.sin(latitude*math.pi/180)
    y=0.5-math.log((1 + sinLatitude) / (1 - sinLatitude))/(4*math.pi)
    
    mapSize=MapSize(levelofDetail)
    pixelX=Clip(x*mapSize+0.5,0,mapSize-1)
    pixelY=Clip(y*mapSize+0.5,0,mapSize-1)
    return int(pixelX),int(pixelY)

def PixelXYtoLatLong(pixelX,pixelY,levelofDetail):
    """
    Converts a pixel from pixel XY coordinates at a specified level of detail
    into latitude/longitude WGS-84 coordinates (in degrees).
    input: pixelX,pixelY,level of detail
    output: latitude,longitude
    """
    mapSize=MapSize(levelofDetail)
    x=(Clip(pixelX,0,mapSize-1)/mapSize)-0.5
    y=0.5- (Clip(pixelY,0,mapSize-1)/mapSize)
    
    latitude=90- 360*math.atan(math.exp(-y*2*math.pi))/math.pi
    longitude=360*x
    return latitude,longitude

def pixelXYToTileXY(pixelX,pixelY):
    """
     Converts pixel XY coordinates into tile XY coordinates of the tile containing
    the specified pixel.
    input :pixelX,pixelY
    output:tileX,tileY
    """
    tileX=pixelX/256
    tileY=pixelY/256
    return int(tileX),int(tileY)

def TileToPixelXY(tileX,tileY):
    
    """Converts tile XY coordinates into pixel XY coordinates of the upper-left pixel
    of the specified tile.
    input: tileX,tileY
    output: pixelX,pixelY
    """
    pixelX=tileX*256
    pixelY=tileY*256
    return int(pixelX),int(pixelY)

def TileXYToQuadKey(tileX,tileY,levelofDetail):
    """
    Converts tile XY coordinates into a QuadKey at a specified level of detail.
    input:tileX,tileYlevel of detail
    output: quadkey string"""
    quadKey=[]
    for i in range(levelofDetail,0,-1):
        digit=0
        mask=1 <<(i-1)
        if (tileX & mask)!=0:
            digit+=1
        if (tileY & mask)!=0:
            digit+=1
            digit+=1
        quadKey.append(str(digit))
    return "".join(quadKey)

def QuadkeyToTileXY(quadKey):
    """
     Converts a QuadKey into tile XY coordinates.
     input: quadkey
     output: tileX,tileY,levelofDetail"""
    tileX=0
    tileY=0
    levelofDetail=len(quadKey)
    for i in range(levelofDetail,0,-1):
        mask=1 <<(i-1)
        q=quadKey[levelofDetail-i]
        if q=='0':
            break
        elif q=='1':
            tileX |=mask
            break
        elif q=='2':
            tileY |=mask
            break
        elif q=='3':
            tileX |=mask
            tileY |=mask
            break
        else:
            print("invalid quadKey")
            

In [4]:
def get_img(lat,long,levelofDetail):
    """ 
    gets the image for given parameters
    input:latitude, longitude,levelofDetail
    output: returns quadkey and stores image
    """
    pixelX,pixelY=LatLongToPixelXY(lat,long,levelofDetail)
    tileX,tileY=pixelXYToTileXY(pixelX,pixelY)
    q=TileXYToQuadKey(tileX,tileY,levelofDetail)
    part1_url=" http://h0.ortho.tiles.virtualearth.net/tiles/h"
    part2_url=".jpeg?g=131"
    rx(part1_url+q+part2_url,"temp.jpeg")
    return q,plt.imread("temp.jpeg")

def find_img(lat,long):
    """finds the higest resolution/levelofDetail image available
    input:latitude,longitude
    output:returns levelofDetail"""
    for i in range(HigestLevelofDetail,1,-1):
        quadkey,img=get_img(lat,long,i)
        if np.array_equal(img,plt.imread("empty.jpeg")):
            {}#find the next lower levelofDetail img
        else:
            cv2.imwrite(str(q)+".jpeg",img)
            break
    return quadkey

lat=41.8781
long=-87.6298
find_img(lat,long)

NameError: name 'HigestLevelofDetail' is not defined

In [35]:
#aurora
lat1=39.7294
long1=-104.8319

#chicago
lat2=41.8781
long2=-87.6298

def order(lat1,long1,lat2,long2):
    """
    orders lat1<lat2 and long1<long2
    input:2 pairs of lat,long cordinates
    output: same cordinates but ordered
    """
    if lat1>lat2:
        temp=lat1
        lat1=lat2
        lat2=temp
    if long1>long2:
        temp=long1
        long1=long2
        long2=temp
    return lat1,long1,lat2,long2

def find_MaxLevelofDetail(lat,long):
    """finds the higest resolution/levelofDetail image available
    input:latitude,longitude
    output:returns depth
    """
    for i in range(MaxLevelofDetail,1,-1):
        quadkey,img=get_img(lat,long,i)
        if np.array_equal(img,plt.imread("empty.jpeg")):
            {}#find the next lower levelofDetail img
        else:
            break       
    return i
    
def get_LevelofDetail(lat1,long1,lat2,long2,part=3):
    """
    divides the bounding box in grids along lat,long to get 
    sample points followed by their max levelofDetail
    input ordered 2 pair of lat,long coordinates & # of partations to make
    output min levelofDetails among all imgs with max levelofdetail for sample points
    """
    f=[]
    delta_x=(lat2-lat1)/part
    delta_y=(long2-long1)/part
    i=lat1
    while i<=lat2:
        j=long1
        while j<=long2:
            f.append(find_MaxLevelofDetail(i,j))
            j+=delta_y
        i+=delta_x
    return min(f)

def img_size(lat1,long1,lat2,long2):
    """finds image size given 2 pair of lat,long coordinates
    input 2 pairs of lat long coordinates
    output image size in KB
    """
    pixelX1,pixelY1=LatLongToPixelXY(lat1,long1,levelofDetail)
    pixelX2,pixelY2=LatLongToPixelXY(lat2,long2,levelofDetail)
    delta_X=abs(pixelX1-pixelX2)
    delta_Y=abs(pixelY1-pixelY2)
    num_of_pixels=delta_X*delta_Y
    #size per pixel is 24 bytes
    return (num_of_pixels*24)/1000
    
def find_box(x1,y1,x2,y2):
    """
    retrive image for given lat,long
    input lat,long of 2 points
    output """
    if x1==x2 or y1==y2:
        print("select coordinates not on same line ")
        return
    else:
        lat1,long1,lat2,long2=order(lat1,long1,lat2,long2)
        maxlevel=get_LevelofDetail(lat1,long1,lat2,long2)
        #now all tiles retrived will be from same max possible levelofDetail
        
        #checking img size
        if img_size(lat1,long1,lat2,long2,maxlevel)>200:
            print("region too large select smaller region")
        else:
            #retrive tiles
            #crop
            pass